# Analyzing Transient CSS161010

The Coordinates (right ascension and declination) of an astronomical transient discovered at optic al wavelengths are : RA= 04:58:34.409 (hr: minutes: seconds) dec= -08:18:03.98 (deg: arcmin : arcsec)
We observed the transient with Chandra. Retrieve and analyze the Chandra data with ID= 19984. 

Supernova Catalog: https://sne.space/sne/CSS161010:045834-081803/

Data can be retreived from: https://cda.harvard.edu/chaser/mainEntry.do with ID=19984

### Goal:  is this transient detected in the X-rays by Chandra?

In [1]:
from astropy.io import fits
import pandas as pd
import matplotlib.pyplot as plt
from astropy.time import Time
import numpy as np
import math
plt.style.use('default')

# ------------------------------------ Functions ------------------------------------

### Read Fits File

In [2]:
def read_fits(fits_file,txt_file,column): 
    hdulist = fits.open(fits_file)
    #hdulist.info() 
    tbdata = hdulist[1].data
    
    f=open(txt_file,'w+')
    
    c=[]
    for i in range(len(column)): 
        c.append(tbdata.field(column[i])) 

    #f.write(column_head)
    print (len(c[1]))

    if len(column) == 2:
        for i in range(len(c[0])):
            f.write('%e,%e\r\n'%(c[0][i],c[0][i]))
                
    if len(column)== 3: 
        for i in range(len(c[0])):
          
            f.write('%e,%e,%e\r\n'%(c[0][i],c[1][i],c[2][i]))

    if len(column)== 4: 
        for i in range(len(c[0])):
          
            f.write('%e,%e,%e,%e\r\n'%(c[0][i],c[1][i],c[2][i],c[3][i]))
                

    df = pd.read_csv(txt_file ,sep=",", header=None, names=column)

    hdulist.close()
    
    return df

### Read in Fits File and Convert to Text

In [3]:
# read_fits(fits_file= '19984/primary/acisf19984N002_evt2.fits',
#           txt_file = '19984_source.txt', 
#           column = ['detx','dety','energy']) 

### Count Photons in a Region

In [4]:
def ph_count(region_type,textfile,x_pos,y_pos,reg,e1,e2):
    coord_list = []
    with open(textfile) as f:
         for row in f:
            if not row.startswith("#"):
                coord_list.append(map(float, row.split(',')))
    points = [(x,y,z) for (x,y,z) in coord_list if ((x-x_pos)**2 + (y-y_pos)**2) <= reg**2 and e1<=z<=e2]
    count = len(points)
    
    text = print ('Photon Count in', region_type ,'region with energy',(e1/1000),'-',(e2/1000),'keV' ,'Energy Contraint = ', count,' photons') 
    return count,text 

###  POISSON STATISTICS: Photons w 3$\sigma$ detection assuming
$e^{-y}  \sum_{0}^{x} \frac{y^{x}}{x!} > P $ where P =  1-.003 = .997 to get detection to 3sigma

In [5]:
def poisson(k,y): # k = source, y=expected
    def sigma_sum(start, end, expression):
        return sum(expression(i) for i in range(start, end))

    summ = []
    summ.append(sigma_sum(0, k, lambda k: np.exp(-y)*(y**k)/(math.factorial(k)) )) 
    summ = (summ[0]) 

    sum_rounded = round(summ,3) 

    if sum_rounded >=  .997:
        tf = ('DETECTION -- p = 0.997 < or = limit (' +(str(sum_rounded)) +')') 
    else:
        tf =  ('NO DETECTION -- p = 0.997 > limit (' +(str(sum_rounded))+')') 
    

    return tf,sum_rounded

### Minimum Number of Photons Needed for 3$\sigma$ Detection

In [6]:
def min_phot(y,xmax):
    k_val = []
    for k in range(0,xmax): #maximum amount of x values you want to go through 
        if poisson(k,y)[1] >= .997:
           k_val.append(k)
        
    k = k_val[0]
    min_photons =print ('Minimum number of photons to claim a 3sigma detection: ' + str(k) +' photons') 
    return min_photons

# ------------------------------------- Analysis -------------------------------------

START (UT): 2017-01-13T23:13:05

STOP (UT) : 2017-01-14T07:59:33

Exposure(KS:2.965E4 ks

## -------------------------------------------- .5-8 keV -------------------------------------------- 

In [7]:
bckg_reg = 61 #arcsec, a region much larger than the source region, but not close the edge of any of the ccds
src_reg  = 6.1 #arcsec

#### a) Filter Energy for Background Region

In [8]:
bkg = ph_count('background','19984_source.txt',4067.1958,4090.2618,bckg_reg,.5e3,8e3) 

Photon Count in background region with energy 0.5 - 8.0 keV Energy Contraint =  224  photons


#### b) Actual Photons in Source Region

In [9]:
actual_src = ph_count('source','19984_source.txt',4067.1958,4090.2618,src_reg,.5e3,8e3)[0]

Photon Count in source region with energy 0.5 - 8.0 keV Energy Contraint =  3  photons


#### c) Expected Photons Source Region

In [10]:
norm_bkg = (bkg[0]) * ((src_reg**2)/(bckg_reg**2)) 
print ('expected source from background = ',norm_bkg,'photons')

expected source from background =  2.2399999999999998 photons


#### d) Minimum Amount of Photons to Get Detection

In [11]:
min_phot(norm_bkg,50)

Minimum number of photons to claim a 3sigma detection: 8 photons


#### e) Detection?

In [12]:
poisson(actual_src,norm_bkg)[0]

'NO DETECTION -- p = 0.997 > limit (0.612)'

## -------------------------------------------- 1-3 keV -------------------------------------------- 
#### a) Filter Energy for Background Region

In [13]:
bkg = ph_count('background','19984_source.txt',4067.1309,4090.2618,bckg_reg,1e3,3e3)

Photon Count in background region with energy 1.0 - 3.0 keV Energy Contraint =  72  photons


#### b) Actual Photons in Source Region

In [14]:
actual_src = ph_count('source','19984_source.txt',4067.1309,4090.2618,src_reg,1e3,3e3)[0]

Photon Count in source region with energy 1.0 - 3.0 keV Energy Contraint =  1  photons


#### c) Expected Phtons in Source Region

In [15]:
norm_bkg = (bkg[0]/bckg_reg**2) *(src_reg**2)
print ('expected source from background = ',norm_bkg,'photons')

expected source from background =  0.72 photons


#### d) Minimum Amount of Photons to Get Detection


In [16]:
min_phot(norm_bkg,50)

Minimum number of photons to claim a 3sigma detection: 5 photons


#### e) Detection?

In [17]:
poisson(actual_src,norm_bkg)[0]

'NO DETECTION -- p = 0.997 > limit (0.487)'